In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [2]:
url_base='http://www.csmpt.org.tw/news'
url_work_table='/index.php?type=4'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [3]:
pages_link=[]
for o in soup.find(class_="page").find_all('span'):
    if o.find('a'):
        O=o.find("a")
        if O.text == "到最後>|" or O.text == ">":
            pass
        else:
            link = url.replace("?type=4","")+O.get('href')
            pages_link.append(link)
            print(O.text,link)

# Get work table

The `get_work_page` function is used for getting each work page

In [4]:
work_page = soup.find_all('article')[0]

In [5]:
def get_work_page(page_url,page=None,page_link_part='?page='):
    global url
    if page != None:
        page_url = page_url+page_link_part+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

Using `get_work_page` to read each page and using `get_tables_part` read work table

In [6]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [11]:
def get_work_deadtime(link):
    y=get_work_detail(link).find('div', class_="content").text.replace('\xa0','').replace("\n","").replace(" 年","年").replace(" 月","月").replace(" 日","日")
    date_pattern="(\d{1,4}年)((([0?][1-9])月)|(([1?][0-2])月)|([1-9]月)?)(([0?][1-9]日)|([1?][0-9]日)|([2?][1-9]日)|([3][0-1]日)?)"
    return re.search(date_pattern,re.search("(?<=[期限,截止])(.*)年(.*)月(.*)日",y)[0]).group(0)

In [8]:
def get_work_table(table_,work_table):
    for p_i, p_item in enumerate(table_.find_all('li')):
        if p_item.find('a'):
            p_item_a = p_item.find('a')
            title = p_item_a.text
            link_s = url_base+'/'+p_item_a.get('href')
            try:
                dead_line = get_work_deadtime(link_s)
            except:
                dead_line = "null(check webpage)"
            print(p_i-1,title,link_s,dead_line)
            work_table.append([p_i-1, title, dead_line, link_s ])
    return work_table

In [12]:
work_table=[]
work_table = get_work_table(work_page,work_table)
for p_i, p_item in enumerate(pages_link):
    table_ = get_work_page(p_item)

    work_table = get_work_table(table_,work_table)


1 永康奇美醫院 放射腫瘤科誠徵醫學物理師一名 http://www.csmpt.org.tw/news/content.php?id=363&type=4 2023年2月10日
2 瓦里安台灣招聘需求 - Clinical Marketing Specialist 臨床市場專員 http://www.csmpt.org.tw/news/content.php?id=357&type=4 null(check webpage)
3 2 Med Phys Positions open in Auckland, NZ http://www.csmpt.org.tw/news/content.php?id=355&type=4 null(check webpage)
4 臺北醫學大學附設醫院放射腫瘤科徵醫學物理師 http://www.csmpt.org.tw/news/content.php?id=354&type=4 null(check webpage)
5 臺北榮民總醫院 (腫瘤醫學部放射腫瘤科)徵定期契約醫事放射師(醫學物理師) http://www.csmpt.org.tw/news/content.php?id=345&type=4 111年
6 後山花蓮門諾醫院徵醫學物理師 http://www.csmpt.org.tw/news/content.php?id=343&type=4 null(check webpage)
7 合富醫療集團誠徵資深物理師 http://www.csmpt.org.tw/news/content.php?id=341&type=4 null(check webpage)
8 Precess Medical Derivative http://www.csmpt.org.tw/news/content.php?id=333&type=4 null(check webpage)


In [13]:
work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

   no                                              召聘職稱                   期限  \
0   1                             永康奇美醫院 放射腫瘤科誠徵醫學物理師一名           2023年2月10日   
1   2  瓦里安台灣招聘需求 - Clinical Marketing Specialist 臨床市場專員  null(check webpage)   
2   3         2 Med Phys Positions open in Auckland, NZ  null(check webpage)   
3   4                             臺北醫學大學附設醫院放射腫瘤科徵醫學物理師  null(check webpage)   
4   5             臺北榮民總醫院 (腫瘤醫學部放射腫瘤科)徵定期契約醫事放射師(醫學物理師)                 111年   
5   6                                    後山花蓮門諾醫院徵醫學物理師  null(check webpage)   
6   7                                     合富醫療集團誠徵資深物理師  null(check webpage)   
7   8                        Precess Medical Derivative  null(check webpage)   

                                                  連結  
0  http://www.csmpt.org.tw/news/content.php?id=36...  
1  http://www.csmpt.org.tw/news/content.php?id=35...  
2  http://www.csmpt.org.tw/news/content.php?id=35...  
3  http://www.csmpt.org.tw/news/content.php?id=35...  
4  h